In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#A cleaned version Brazil MST data (only for 2005)
data = pd.read_stata('C:/Users/user/Desktop/Advanced Data Analysis with Python 9.9.22/Datasets/Brazil MST data 2005.dta')
data.columns

### Installation of Geopandas

In [ ]:
#pip install geopandas

If you get trouble while installing geopandas, follow the conda way. Please follow the conda section in the following link: https://geopandas.org/en/stable/getting_started/install.html

Summary, run following codes in anaconda prompt, give your permission if asked:

1. conda install --channel conda-forge geopandas

2. conda create -n geo_env

3. conda activate geo_env

4. conda config --env --add channels conda-forge

5. conda config --env --set channel_priority strict

6. conda install python=3 geopandas


Then, activate geo_env in the anaconda prompt, and call "jupyter notebook" in that environment. Then you are using your notebook in a separate place. Note that, python might ask you to install some packages again, you are in a new zone.

In [ ]:
import geopandas

### Read, merge, and plot

In [ ]:
import geopandas as gpd

shapefile = gpd.read_file("BR/BRMUE250GC_SIR.shp")
print(shapefile)

In [ ]:
shapefile.plot()
#we don't have any varaible yet, only ibgecode and geometry

In [ ]:
#Let's merge them
merged_data = shapefile.merge(data, on='ibgecode')

In [ ]:
merged_data.plot()

In [ ]:
merged_data.boundary

In [ ]:
merged_data.centroid

In [ ]:
print(merged_data.head())

In [ ]:
# Check what kind of object merged_data is
print(type(merged_data))

In [ ]:
# Check the type of the geometry attribute
print(type(merged_data.geometry))

In [ ]:
# Inspect the first rows of the geometry
print(merged_data.geometry.head())

In [ ]:
# Inspect the area of the municipalities
print(merged_data.geometry.area)

In [ ]:
#Plot by invasion count
merged_data.plot(column='invasions_count_cum')

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1)
merged_data.plot(column='log_income', ax=ax, legend=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
merged_data.plot(column='log_income', cmap='OrRd', scheme='quantiles', ax=ax, legend=True)
ax.set_axis_off()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
merged_data.plot(column='log_income', ax=ax, legend=True, 
                 legend_kwds={'label': "Income by municipality", 'orientation': "horizontal"})

In [ ]:
merged_data.boundary.plot(figsize=(10, 10))

In [ ]:
invasions = merged_data[merged_data['invasions_dum'] == 1]

In [ ]:
inv_plot = invasions.plot(marker='*', color='red', markersize=5, figsize=(10, 10))
inv_plot.set_axis_off()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
merged_data.plot(ax=ax, color='black')

invasions.plot(ax=ax, color='white')

ax.set_axis_off()
plt.show()

In [ ]:
base = merged_data.plot(color='black', figsize=(10, 10))
invasions.plot(ax=base, marker='o', color='white', markersize=5)

In [ ]:
base = merged_data.plot(color='black', edgecolor='green', figsize=(10, 10))
invasions.plot(ax=base, marker='o', color='purple', markersize=5)

### Spatial weight matrix

In [ ]:
#pip install pysal

1. conda install --channel conda-forge pysal

(skip second step if you created the geo_env above)

2. conda create -n geo_env

3. conda activate geo_env

4. conda config --env --add channels conda-forge

5. conda config --env --set channel_priority strict

6. conda install python=3 pysal

In [ ]:
import pysal as ps

In [ ]:
import libpysal

In [ ]:
from libpysal.weights import Queen, Rook, KNN

Queen: More inclusive, catches any vertex

Rook: Considers edges

KNN: Equal number of neighbors for each, according to distances

In [ ]:
w_knn1 = KNN.from_shapefile('BR/BRMUE250GC_SIR.shp')

In [ ]:
len(w_knn1.neighbors)

In [ ]:
w_knn1.neighbors

In [ ]:
w_knn3 = KNN.from_shapefile('BR/BRMUE250GC_SIR.shp',3) #default was 2

In [ ]:
w_knn3.neighbors

In [ ]:
w_queen = Queen.from_shapefile('BR/BRMUE250GC_SIR.shp')

In [ ]:
len(w_queen.neighbors)

In [ ]:
w_queen.neighbors

In [ ]:
w_rook = Rook.from_shapefile('BR/BRMUE250GC_SIR.shp')

In [ ]:
len(w_rook.neighbors)

In [ ]:
w_rook.neighbors

In [ ]:
w_rook.neighbors[4]

In [ ]:
w_queen.neighbors[4]

In [ ]:
#We can call weight object directly from the dataframe
wq = libpysal.weights.Queen.from_dataframe(merged_data)
wq

In [ ]:
#We need to standardize our weights with respect to rows
wq.transform = 'r'

### Produce spatial lags

In [ ]:
merged_data['invasions_count_splag'] = libpysal.weights.lag_spatial(wq, merged_data['invasions_count'])

In [ ]:
merged_data['invasions_count_splag'],merged_data['invasions_count']

In [ ]:
merged_data['invasions_count'].corr(merged_data['invasions_count_splag'])

In [ ]:
merged_data['BF_coverage1000_splag'] = libpysal.weights.lag_spatial(wq, merged_data['BF_coverage1000'])
merged_data['BF_coverage1000'].corr(merged_data['BF_coverage1000_splag'])

!!! Spatial autocorrelation alert !!!

In [ ]:
merged_data['log_income_splag'] = libpysal.weights.lag_spatial(wq, merged_data['log_income'])
merged_data['ln_agriculturalproductivity_splag'] = libpysal.weights.lag_spatial(wq, merged_data['ln_agriculturalproductivity'])
merged_data['PT_voteshare_splag'] = libpysal.weights.lag_spatial(wq, merged_data['PT_voteshare'])
merged_data['infantmortality_splag'] = libpysal.weights.lag_spatial(wq, merged_data['infantmortality'])

import seaborn as sns

fig = plt.subplots(figsize=(15, 15))
sns.heatmap(merged_data[['invasions_count', 'invasions_count_splag', 'log_income', 'log_income_splag', 
                        'ln_agriculturalproductivity','ln_agriculturalproductivity_splag', 'PT_voteshare',
                        'PT_voteshare_splag','infantmortality','infantmortality_splag']].corr(), cmap="YlGnBu", annot=True)
plt.show()

### Run the Spatial regression

http://darribas.org/gds_scipy16/ipynb_md/08_spatial_regression.html

In [ ]:
merged_data = merged_data.dropna(axis=0)

In [ ]:
#pip install spreg

In [ ]:
import spreg

In [ ]:
#spreg only works with numpy arrays

In [ ]:
y = merged_data['BF_coverage1000'].values

In [ ]:
x_OLS = merged_data[['invasions_count_cum', 'log_income', 'ln_agriculturalproductivity', 'PT_voteshare','infantmortality']].values
x_OLS

In [ ]:
#We can redefine a weighting matrix, reading neighbors and weights
#from libpysal.weights import W
#w_matrix = W(wq.neighbors, wq.weights)

In [ ]:
#We dropped some observations, then we need to recalculate the weights, to keep the total size consistent 
wq = libpysal.weights.Queen.from_dataframe(merged_data)
wq.transform = 'r'

##### OLS with spreg

In [ ]:
model_OLS = spreg.OLS(
    y[:, None],
    x_OLS,
    w=wq,
    spat_diag=True,
    #name_x=['invasions_count_cum', 'log_income', 'ln_agriculturalproductivity', 'PT_voteshare','infantmortality'], 
    #name_y='Bolsa Familia'
)

print(model_OLS.summary)

DIAGNOSTICS FOR SPATIAL DEPENDENCE: The main summary from the diagnostics for spatial dependence is that there is clear evidence to reject the null of spatial randomness in the residuals, hence an explicitly spatial approach is warranted.

##### Including Spatial lag of X

In [ ]:
x_splag_X = merged_data[['invasions_count_cum', 'invasions_count_splag', 'log_income', 'ln_agriculturalproductivity', 'PT_voteshare','infantmortality']].values

In [ ]:
model_splag_of_X = spreg.OLS(
    y[:, None],
    x_splag_X,
    w=wq,
    spat_diag=True,
    #name_x=['invasions_count_cum', 'invasions_count_splag', 'log_income', 'ln_agriculturalproductivity', 'PT_voteshare','infantmortality'], 
    #name_y='Bolsa Familia'
)

print(model_splag_of_X.summary)

#### Try OLS for the same model:

In [ ]:
import statsmodels.api as sm
x_splag_X1 = sm.add_constant(x_splag_X)
model = sm.OLS(y, x_splag_X1)
model_result = model.fit()
model_result.summary()

##### Including Spatial lag of Y

In [ ]:
x_splag_Y = merged_data[['invasions_count_cum', 'BF_coverage1000_splag', 'log_income', 'ln_agriculturalproductivity', 'PT_voteshare','infantmortality']].values

In [ ]:
model_splag_of_Y = spreg.OLS(
    y[:, None],
    x_splag_Y,
    w=wq,
    spat_diag=True,
    #name_x=['invasions_count_cum', 'BF_coverage1000_splag', 'log_income', 'ln_agriculturalproductivity', 'PT_voteshare','infantmortality'], 
    #name_y='Bolsa Familia'
)

print(model_splag_of_Y.summary)

##### Including both spatial lags

In [ ]:
x_splags = merged_data[['invasions_count_cum', 'invasions_count_splag', 'BF_coverage1000_splag', 'log_income', 'ln_agriculturalproductivity', 'PT_voteshare','infantmortality']].values

In [ ]:
model_splags_both= spreg.OLS(
    y[:, None],
    x_splags,
    w=wq,
    spat_diag=True,
    #name_x=['invasions_count_cum', 'invasions_count_splag', 'BF_coverage1000_splag', 'log_income', 'ln_agriculturalproductivity', 'PT_voteshare','infantmortality'], 
    #name_y='Bolsa Familia'
)

print(model_splags_both.summary)

!!!! Let's compare results !!!!

In [ ]:
from sklearn.metrics import mean_squared_error as mse

mses = pd.Series({'OLS': mse(y, model_OLS.predy.flatten()),
                     'splag_of_X': mse(y, model_splag_of_X.predy.flatten()),
                     'splag_of_Y': mse(y, model_splag_of_Y.predy.flatten()),
                     'both_splags': mse(y, model_splags_both.predy.flatten())
                    })
mses.sort_values()

### IV2sls with spatial lags

In [ ]:
#If you are working in geo_env, your environment probably does not have "linearmodels" package. Then pip it!
#pip install linearmodels

In [ ]:
type(merged_data)

In [ ]:
#Conver geopandas dataframe into a pandas dataframe object
merged_data_pd = pd.DataFrame(merged_data)

In [ ]:
type(merged_data_pd)

In [ ]:
import linearmodels
from linearmodels.iv import IV2SLS
sp_formula = 'BF_coverage1000 ~ 1 + log_income + ln_agriculturalproductivity + PT_voteshare + infantmortality + BF_coverage1000_splag + invasions_count_splag + [log_invasions_count_cum ~ log_intended_lands1995]'
iv2sls = IV2SLS.from_formula(sp_formula, merged_data_pd).fit()
iv2sls